In [244]:
import numpy as np
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [245]:
#wikipedia wrapper will only fetch 1st top result and only 200 characters
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)


In [246]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [251]:
LANGCHAIN_API_KEY= 'xxxxx'
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [252]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_mistralai import MistralAIEmbeddings

In [278]:
import os
api_key = "xxxxxx"
Model = ChatMistralAI(api_key=api_key, model=model,temperature = 0.7)

In [279]:
#we are using mistralAi for embedding for our documents and MistralAI for our
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=api_key)
loader = WebBaseLoader("https://education.nationalgeographic.org/resource/mining/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = documents.split_documents(docs)

/Users/pookie/PycharmProjects/AI_agent/.venv/lib/python3.13/site-packages/langchain_mistralai/embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [280]:
vectordb = FAISS.from_documents(documents, embedding = embeddings)

In [258]:
web_retriever = vectordb.as_retriever()
len(documents) #No of chunks made

6

In [259]:
from langchain.tools.retriever import create_retriever_tool
#Making a retriever tool which can be used to retrieve relevant text chunks based on the query using vector similarity search
web_retriever_tool=create_retriever_tool(web_retriever,"Mining_Article","For any queries regarding Mining use this article !")

In [260]:
web_retriever_tool.name

'Mining_Article'

In [261]:
import numpy as np
#having a look at the first_embedding of dim 1024 stroed in faiss vectordabase
first_embedding = np.array(vectordb.index.reconstruct(0))

print("Embedding for the first document chunk:")
print(first_embedding)
#storing the embeddings locally
vectordb.save_local("faiss_index")  # Saves embeddings locally


Embedding for the first document chunk:
[-0.00465393  0.04071045  0.02700806 ...  0.0017395   0.02641296
 -0.04760742]


In [262]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/Users/pookie/PycharmProjects/AI_agent/AI_agents /Astro-economics.pdf")
docs1 = loader.load()
documents1= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents1 = documents1.split_documents(docs1)
len(documents1)

186

In [263]:
#there is a lot of chunks in documents1 so we will only consider first 15 for simplicity
vectordb = FAISS.from_documents(documents1[:15], embedding= embeddings)
pdf_retriever = vectordb.as_retriever()

In [264]:
#Creating a pdf retriever tool
pdf_retriever_tool = create_retriever_tool(pdf_retriever,"Astro_economics","for any query related to astro-economics use this article !")

In [265]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

#Using Arxiv API to search academic papers from ArXiv
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
#will only give top 1 paper and max_character = 200
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [266]:
tools=[wiki,arxiv,web_retriever_tool,pdf_retriever_tool]

In [267]:
from langchain import hub
#We use the prompts from langchain hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [268]:
from langchain.agents import  create_openai_tools_agent
agent= create_openai_tools_agent(Model,tools,prompt)

In [269]:
#Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent= agent , tools = tools, verbose=True )
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [270]:
agent_executor.invoke({"input":"Who has written the article on Astro-economics? "})



> Entering new AgentExecutor chain...

Invoking: `Astro_economics` with `{'query': 'Author of the article'}`


See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/381574882
ASTRO-ECONOMICS: UNRAVELING THE FINANCIAL DIMENSIONS OF SPACE
ENDEAVORS
Article · June 2024
CITATIONS
0
READS
281
5 authors, including:
Bhushan Satish Deshmane
Indian Institute of Technology Kharagpur
1 PUBLICATION   0 CITATIONS   
SEE PROFILE
Animesh Raj
Indian Institute of Technology Kharagpur
1 PUBLICATION   0 CITATIONS   
SEE PROFILE
Annu Chaurasiya
Indian Institute of Technology Kharagpur
1 PUBLICATION   0 CITATIONS   
SEE PROFILE
Dakshita Pokharna
Indian Institute of Technology Kharagpur
1 PUBLICATION   0 CITATIONS   
SEE PROFILE
All content following this page was uploaded by Heramb Warke on 20 June 2024.
The user has requested enhancement of the downloaded file.

and identify pollution transfers between 
different environmental impacts or life-cycle

{'input': 'Who has written the article on Astro-economics? ',
 'output': 'The article on astro-economics has been authored by Bhushan Satish Deshmane, Animesh Raj, Annu Chaurasiya, and Dakshita Pokharna all from the Indian Institute of Technology Kharagpur. For more details, you can visit the publication at [ResearchGate](https://www.researchgate.net/publication/381574882).'}

In [243]:
agent_executor.invoke({"input":"Mining creates a type of water pollution known as?"})



> Entering new AgentExecutor chain...

Invoking: `Mining_Article` with `{'query': 'Mining creates a type of water pollution known as?'}`


diseases that cause breathing impairment and can be fatal.The mining process can also harm the environment in other ways. Mining creates a type of water pollution known as acid mine drainage. First, mining exposes sulfides in the soil. When the rainwater or streams dissolves the sulfides, they form acids. This acidic water damages aquatic plants and animals. Along with acid mine drainage, the disposal of mine waste can also cause severe water pollution from toxic metals. The toxic metals commonly found in mine waste, such as arsenic and mercury, are harmful to the health of people and wildlife if they are released into nearby streams.CreditsMedia CreditsThe audio, illustrations, photos, and videos are credited beneath the media asset, except for promotional images, which generally link to another page that contains the media credit. The Rights Hol

{'input': 'Mining creates a type of water pollution known as?',
 'output': 'Mining creates a type of water pollution known as acid mine drainage'}

In [271]:
agent_executor.invoke({"input":"find a research paper on Space Tourism"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Space Tourism'}`


Published: 2024-06-24
Title: A Carrying Capacity Calculator for Pedestrians Using OpenStreetMap Data: Application to Urban Tourism and Public Spaces
Authors: Duarte Sampaio de Almeida, Rodrigo Simões,The research paper titled "A Carrying Capacity Calculator for Pedestrians Using OpenStreetMap Data: Application to Urban Tourism and Public Spaces" was published on June 24, 2024. The authors of this paper are Duarte Sampaio de Almeida and Rodrigo Simões. The paper presents a methodology for calculating the carrying capacity of pedestrian spaces using OpenStreetMap data, with applications in urban tourism and public space management.

> Finished chain.


{'input': 'find a research paper on Space Tourism',
 'output': 'The research paper titled "A Carrying Capacity Calculator for Pedestrians Using OpenStreetMap Data: Application to Urban Tourism and Public Spaces" was published on June 24, 2024. The authors of this paper are Duarte Sampaio de Almeida and Rodrigo Simões. The paper presents a methodology for calculating the carrying capacity of pedestrian spaces using OpenStreetMap data, with applications in urban tourism and public space management.'}